In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import os, datetime
%matplotlib inline 

# Global Variables
RES=128
NB_CLASSES = 4
BATCH_SIZE=16
EPOCHS=100
OPT = tf.keras.optimizers.RMSprop()

# Evaluate on test dataset helper function
def eval_help(model, x_test, y_test):
    score = model.evaluate(x_test_cn, y_test_cn, batch_size=BATCH_SIZE, verbose=1)
    print(f"Test Score: {score[0]}")
    print(f"Test Accuracy: {score[1]}")
    return score[1]

# Get images
def get_data(directory):
    x = []
    y = []
    mapping={'no_tumor':0, 'pituitary_tumor':1, 'meningioma_tumor':2, 'glioma_tumor':3}
    count=0
    for file in os.listdir(directory):
        path=os.path.join(directory,file)
        for im in os.listdir(path):
            image=load_img(os.path.join(path,im), grayscale=True, color_mode="grayscale", target_size=(RES,RES))
            image=img_to_array(image)
            image = image/255.0 #normalize
            x.append(image)
            y.append(count)  
        count=count+1
    return (x,y)

In [3]:
# Get the training and testing data
(x_train, y_train) = get_data('/kaggle/input/brain-tumor-classification-mri/Training')
(x_test, y_test) = get_data('/kaggle/input/brain-tumor-classification-mri/Testing')

# Change to np array
x_train_cn = np.asarray(x_train).reshape(len(x_train), RES, RES, 1)
x_test_cn = np.asarray(x_test).reshape(len(x_test), RES, RES, 1)
y_train_cn = np.asarray(y_train)
y_test_cn = np.asarray(y_test)

# Change to float32 to increase precision
x_train_cn = x_train_cn.astype('float32')
x_test_cn = x_test_cn.astype('float32')

# normalize data
x_train_cn = x_train_cn / 255.0
x_test_cn = x_test_cn / 255.0

# One hot encode labels
y_train_cn = tf.keras.utils.to_categorical(y_train_cn, NB_CLASSES)
y_test_cn = tf.keras.utils.to_categorical(y_test_cn, NB_CLASSES)

# Shuffle data in training set

# Split training data inot training and validation
x_train_split, x_validation_split, y_train_split, y_validation_split = train_test_split(x_train_cn, y_train_cn, shuffle=True, random_state=101)
    # Good idea to check the counts of the classe in training and validation set to see releative balance
    
    
# Create shallow arch
def shallow_covnet_arch():
    model = models.Sequential()
    model.add(layers.Convolution2D(32, (3,3), activation='relu', input_shape=(RES, RES, 1)))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2))
    
    
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(NB_CLASSES, activation='softmax'))
    return model


# Create image data gen
imggen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=0,
        zoom_range = 0,
        width_shift_range=0,  
        height_shift_range=0,  
        horizontal_flip=True,  
        vertical_flip=False)


# Define early stopper callback so we dont waste time
es = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience = 3
)

In [7]:
# fit imggen to x_train_split
imggen.fit(x_train_split)

# create model
model = shallow_covnet_arch()

# Compile Model
model.compile(optimizer = OPT , loss = "categorical_crossentropy", metrics=["accuracy"])

# fit model to imggen on x_train_split and y_train_split
history = model.fit_generator(imggen.flow(x_train_split,y_train_split,batch_size = BATCH_SIZE),
                              epochs = EPOCHS, validation_data = (x_validation_split,y_validation_split),
                              steps_per_epoch = x_train_split.shape[0] // BATCH_SIZE,)   

In [8]:
eval_help(model, x_test, y_test)

In [10]:
model.summary()

In [14]:
model.save_weights('52%acc_model.h5', save_format='h5')

In [16]:
# fit imggen to x_train_split
imggen.fit(x_train_split)

# create model
model = shallow_covnet_arch()

# Compile Model
model.compile(optimizer = OPT , loss = "categorical_crossentropy", metrics=["accuracy"])

# fit model to imggen on x_train_split and y_train_split
history = model.fit_generator(imggen.flow(x_train_split,y_train_split,batch_size = BATCH_SIZE),
                              epochs = 100, validation_data = (x_validation_split,y_validation_split),
                              steps_per_epoch = x_train_split.shape[0] // BATCH_SIZE,)   

In [17]:
eval_help(model, x_test, y_test)

In [18]:
model.save_weights('63%acc_model.h5', save_format='h5')

In [19]:
# fit imggen to x_train_split
imggen.fit(x_train_split)

# create model
model = shallow_covnet_arch()

# Compile Model
model.compile(optimizer = OPT , loss = "categorical_crossentropy", metrics=["accuracy"])

# fit model to imggen on x_train_split and y_train_split
history = model.fit_generator(imggen.flow(x_train_split,y_train_split,batch_size = BATCH_SIZE),
                              epochs = 300, validation_data = (x_validation_split,y_validation_split),
                              steps_per_epoch = x_train_split.shape[0] // BATCH_SIZE,)   

In [21]:
eval_help(model, x_test, y_test)